In [1]:
import os
import pandas as pd

In [2]:
path ='/home/coder/agent-bootcamp/src/2_frameworks/3_code_interpreter/transactions-fraud-datasets/versions/1'
os.listdir(path)

['cards_data.csv',
 'mcc_codes.json',
 'users_data.csv',
 'transactions_data.csv',
 'train_fraud_labels.json']

In [3]:
mcc_codes = pd.read_json(
    os.path.join(path, "mcc_codes.json"),
    orient="index"
)
mcc_codes.reset_index(inplace=True)
mcc_codes.columns = ["mcc_code", "description"]

print(mcc_codes.head())

   mcc_code                           description
0      5812         Eating Places and Restaurants
1      5541                      Service Stations
2      7996  Amusement Parks, Carnivals, Circuses
3      5411          Grocery Stores, Supermarkets
4      4784                 Tolls and Bridge Fees


In [4]:
users_data= pd.read_csv(os.path.join(path, 'users_data.csv'))
users_data.head(2)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5


In [5]:
cards_data = pd.read_csv(os.path.join(path, 'cards_data.csv'))
cards_data.head(2)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No


In [6]:
transactions_data = pd.read_csv(os.path.join(path, 'transactions_data.csv'), nrows=100000)
transactions_data.head(2)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN


In [7]:
temp  = pd.read_json(os.path.join(path, 'train_fraud_labels.json'))
display(temp.head(2))
print(temp.shape)

,target
10649266,No
23410063,No


(8914963, 1)


In [8]:

temp.reset_index(inplace = True)

In [9]:
temp.columns

Index(['index', 'target'], dtype='object')

In [10]:
temp = temp.rename(columns={'index':'id'})
transactions_data = transactions_data.merge (temp, on  = 'id', how ='left')

In [11]:
del temp

In [13]:
users_data = users_data.rename(columns = {'id': 'client_id'}) 
transactions_data1  = transactions_data.merge(users_data, left_on ='client_id', right_on='client_id', validate ='m:1' )

In [14]:
transactions_data1.shape[1]-transactions_data.shape[1]-users_data.shape[1]

-1

In [15]:
'client_id' in transactions_data1.columns

True

In [16]:
transactions_data1.columns

Index(['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'target', 'current_age', 'retirement_age', 'birth_year',
       'birth_month', 'gender', 'address', 'latitude', 'longitude',
       'per_capita_income', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards'],
      dtype='object')

In [17]:
cards_data = cards_data.rename(columns={'id':'card_id'})

In [18]:

cards_data = cards_data.drop('client_id', axis = 1)

In [19]:

transactions_data2  = transactions_data1.merge(cards_data, left_on ='card_id', right_on='card_id', validate ='m:1' )

In [20]:
transactions_data2.shape[1]-transactions_data1.shape[1]-cards_data.shape[1]

-1

In [21]:

transactions_data2.columns

Index(['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'target', 'current_age', 'retirement_age', 'birth_year',
       'birth_month', 'gender', 'address', 'latitude', 'longitude',
       'per_capita_income', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'has_chip', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')

In [22]:

transactions_data2.target.value_counts(dropna=False)

target
No     67124
NaN    32769
Yes      107
Name: count, dtype: int64

In [23]:

bad = transactions_data2[transactions_data2.target=='Yes']
print (bad.shape)
good = transactions_data2[transactions_data2.target=='No']
print (good.shape)

(107, 37)
(67124, 37)


In [24]:
transactions_data3  = pd.concat ([bad, good.sample (10000)], axis =0)

In [ ]:
transactions_data3.shape

(10107, 37)

In [26]:
transactions_data3.to_csv ('/home/coder/agent-bootcamp/tests/tool_tests/example_files/data_b.csv', index  =False)

In [27]:
transactions_data3.target.value_counts(dropna=False)

target
No     10000
Yes      107
Name: count, dtype: int64

In [29]:
pd.set_option('display.max_columns',None)
transactions_data3.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,target,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
3459,7479444,2010-01-01 23:56:00,126,5497,$0.19,Online Transaction,90999,ONLINE,NaN,NaN,4722,NaN,Yes,63,65,1956,10,Male,5159 Wessex Avenue,34.72,-92.35,$13047,$26600,$0,799,4,Mastercard,Debit,5110177937842500,04/2023,932,YES,1,$13555,11/2007,2011,No
9526,7486725,2010-01-03 20:58:00,126,5497,$339.00,Online Transaction,3558,ONLINE,NaN,NaN,3640,NaN,Yes,63,65,1956,10,Male,5159 Wessex Avenue,34.72,-92.35,$13047,$26600,$0,799,4,Mastercard,Debit,5110177937842500,04/2023,932,YES,1,$13555,11/2007,2011,No
12989,7490901,2010-01-04 23:30:00,720,4136,$4.45,Online Transaction,24504,ONLINE,NaN,NaN,4214,NaN,Yes,36,65,1983,5,Female,980 Federal Drive,42.38,-83.10,$9710,$19800,$40038,682,3,Mastercard,Debit,5549447642972376,08/2021,647,YES,1,$974,08/2004,2008,No
13000,7490914,2010-01-04 23:48:00,720,4136,$7.19,Online Transaction,38602,ONLINE,NaN,NaN,5311,NaN,Yes,36,65,1983,5,Female,980 Federal Drive,42.38,-83.10,$9710,$19800,$40038,682,3,Mastercard,Debit,5549447642972376,08/2021,647,YES,1,$974,08/2004,2008,No
13081,7491008,2010-01-05 02:34:00,1644,3444,$8.76,Online Transaction,21776,ONLINE,NaN,NaN,5712,NaN,Yes,83,65,1937,1,Male,637 Seventh Street,34.66,-83.09,$15406,$31943,$1248,686,6,Visa,Credit,4715738219735937,07/2023,643,YES,1,$10900,06/2009,2009,No
